In [1]:
#pip install boto3 pyarrow python-dotenv openpyxl

In [2]:
import os
import pathlib
from dotenv import load_dotenv
import boto3

import pandas as pd
import osc_ingest_trino as osc
import trino
from sqlalchemy.engine import create_engine

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
print("Initializing Dev tables")

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER_USER1'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'demo_dv'
itr_prefix = 'itr_'

sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD_USER1']),
    'http_scheme': 'https',
    'catalog': ingest_catalog,
    'schema': ingest_schema,
}

engine = create_engine(sqlstring, connect_args = sqlargs)
print("connecting with engine " + str(engine))
qres = osc._do_sql(f"show tables in {ingest_schema}", engine, verbose=True)

Initializing Dev tables
connecting with engine Engine(trino://os-climate-user1@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/)
show tables in demo_dv
[]


In [4]:
rename_year_columns={}
for y in range(2000,2100):
    rename_year_columns[str(y)] = 'y{yr}'.format(yr=y)
#rename_year_columns

## Example `credentials.env` file

```
# s3 credentials
S3_ENDPOINT=https://s3.us-east-1.amazonaws.com
S3_BUCKET=ocp-odh-os-demo-s3
S3_ACCESS_KEY=xxx
S3_SECRET_KEY=xxx

# trino credentials
TRINO_USER=xxx
TRINO_PASSWD=xxx
TRINO_HOST=trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org
TRINO_PORT=443
```

In [5]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['DEMO1_S3_ENDPOINT'],
    aws_access_key_id=os.environ['DEMO1_S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['DEMO1_S3_SECRET_KEY'],
)

## load xlsx file

Loading directly from xlsx -> pandas yields cleaner data than attempting multiple translations:
xlsx -> gsheet -> csv -> pandas

In [6]:
# loading excel doesn't work from 'get_object' mode

s3.download_file(
    os.environ['DEMO1_S3_BUCKET'],
    'itr-demo-data/ITR_company_data_minimum_required.xlsx',
    '/tmp/t.xlsx'
)

# load the raw file from the bucket
xls = pd.ExcelFile('/tmp/t.xlsx')
xls.sheet_names

['fundamental_data',
 'projected_target',
 'projected_ei_in_Wh',
 'projected_production']

## ITR Fundamental Data

In [7]:
# this will be output trino table name
tablename = f'{itr_prefix}fundamental_data'

# get this sheet, and assess data types
df = pd.read_excel(xls, f'fundamental_data').convert_dtypes()

# rename columns to forms that sql will handle
osc.enforce_sql_column_names(df, inplace=True)
df.rename(columns=rename_year_columns, inplace=True)

In [8]:
# a way to examine the structure of a pandas data frame
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   company_name              31 non-null     string 
 1   company_id                31 non-null     string 
 2   isic                      0 non-null      Int64  
 3   country                   31 non-null     string 
 4   region                    31 non-null     string 
 5   industry_level_1          0 non-null      Int64  
 6   industry_level_2          0 non-null      Int64  
 7   industry_level_3          0 non-null      Int64  
 8   industry_level_4          0 non-null      Int64  
 9   sector                    31 non-null     string 
 10  company_revenue           30 non-null     Float64
 11  company_market_cap        30 non-null     Float64
 12  company_enterprise_value  30 non-null     Float64
 13  company_total_assets      30 non-null     Float64
 14  company_cash

In [9]:
# For this demonstration example, we just recreate table from scratch each time.
# in live data platform there will need to be policies and mechanisms for either
# appending new data, or overwriting old data, or saving off conditioned by a versioning number
# this is a data governance topic
osc._do_sql(f'drop table if exists {ingest_catalog}.{ingest_schema}.{tablename}', engine, verbose=True)

drop table if exists osc_datacommons_dev.demo_dv.itr_fundamental_data


In [10]:
# generate a sql schema that will correspond to the data types
# of columns in the pandas DF

df.to_sql(tablename, engine, schema=ingest_schema,
          if_exists='append', index=False,
          method=osc.TrinoBatchInsert(batch_size = 15000, verbose = True))

constructed fully qualified table name as: "demo_dv.itr_fundamental_data"
inserting 35 records
  ('Company AG', 'US0079031078', NULL, 'United States', 'North America', NULL, NULL, NULL, NULL, 'Electricity Utilities', 20248547996.81425, 10464805624.288572, 20370723452.973633, 814618.2057245963, 4528467714.726761, 0.4285714285714285)
  ('Company AH', 'US00724F1012', NULL, 'United States', 'North America', NULL, NULL, NULL, NULL, 'Electricity Utilities', 276185899.614351, 170431377.0111033, 348843699.94010025, 27314.648029499283, 69006940.99809252, 0.4285714285714285)
  ('Company AI', 'FR0000125338', NULL, 'France', 'Europe', NULL, NULL, NULL, NULL, 'Electricity Utilities', 10283015131.798985, 3087133686.0634212, 4800604056.581685, 343642.4737086049, 1163119848.4230556, 0.4285714285714285)
  ...
  ('Company AG', 'US0079031078', NULL, 'United States', 'North America', NULL, NULL, NULL, NULL, 'Electricity Utilities', NULL, NULL, NULL, NULL, NULL, NULL)
batch insert result: [(35,)]


In [11]:
# test that we can get data
qres = osc._do_sql(f'select country from {ingest_catalog}.{ingest_schema}.{tablename}', engine, verbose=True)

select country from osc_datacommons_dev.demo_dv.itr_fundamental_data
[('United States',), ('United States',), ('France',), ('United States',), ('Switzerland',), ('United States of America',), ('France',), ('United States of America',), ('Taiwan, Province of China',), ('France',), ('Switzerland',), ('Japan',), ('Japan',), ('United States of America',), ('United Kingdom',), ('United States of America',), ('United States of America',), ('Japan',), ('Netherlands',), ('Italy',), ('Sweden',), ('Sweden',), ('United States of America',), ('China',), ('China',), ('China',), ('Brazil',), ('Brazil',), ('Brazil',), ('Argentinia',), (None,), (None,), (None,), (None,), ('United States',)]


## ITR Projected Production

In [12]:
# this will be output trino table name
tablename = f'{itr_prefix}projected_production'

# get this sheet, and assess data types
df = pd.read_excel(xls, 'projected_production').convert_dtypes()

# rename columns to forms that sql will handle
osc.enforce_sql_column_names(df, inplace=True)
df.rename(columns=rename_year_columns, inplace=True)

In [13]:
# a way to examine the structure of a pandas data frame
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 44 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   company_id  30 non-null     string 
 1   variable    30 non-null     string 
 2   y2009       5 non-null      Float64
 3   y2010       13 non-null     Float64
 4   y2011       13 non-null     Float64
 5   y2012       14 non-null     Float64
 6   y2013       17 non-null     Float64
 7   y2014       24 non-null     Float64
 8   y2015       24 non-null     Float64
 9   y2016       24 non-null     Float64
 10  y2017       26 non-null     Float64
 11  y2018       26 non-null     Float64
 12  y2019       26 non-null     Float64
 13  y2020       26 non-null     Float64
 14  y2021       26 non-null     Float64
 15  y2022       26 non-null     Float64
 16  y2023       26 non-null     Float64
 17  y2024       26 non-null     Float64
 18  y2025       26 non-null     Float64
 19  y2026       26 non-null     Flo

In [14]:
# For this demonstration example, we just recreate table from scratch each time.
# in live data platform there will need to be policies and mechanisms for either
# appending new data, or overwriting old data, or saving off conditioned by a versioning number
# this is a data governance topic
osc._do_sql(f'drop table if exists {ingest_catalog}.{ingest_schema}.{tablename}', engine, verbose=True)

drop table if exists osc_datacommons_dev.demo_dv.itr_projected_production


In [15]:
# generate a sql schema that will correspond to the data types
# of columns in the pandas DF

df.to_sql(tablename, engine, schema=ingest_schema,
          if_exists='append', index=False,
          method=osc.TrinoBatchInsert(batch_size = 15000, verbose = True))

constructed fully qualified table name as: "demo_dv.itr_projected_production"
inserting 30 records
  ('US0079031078', 'Electricity_prod_complete', NULL, NULL, NULL, NULL, NULL, 467435849.836039, 319287605.836039, 375523565.836039, 241767187.636039, 108010809.436039, 104827858.636039, 96880847.84303102, 98800366.55109683, 100757917.04927026, 102754252.86865924, 104790142.47022723, 106866369.54060121, 110273028.01906781, 113788283.07509942, 117415596.52229828, 121158540.5290649, 125020801.13645986, 127708032.7692642, 130453024.5010497, 133257017.8433383, 136121280.99304354, 139047109.40605375, 143803518.1276373, 148722630.1518885, 153810011.09905753, 159071416.97353712, 164512800.67636612, 166864178.74843603, 169249165.01886547, 171668239.84894145, 174121890.46574494, 176610611.06028363, 177180531.38617307, 177752290.83812582, 178325895.3509797, 178901350.87872407, 179478663.39456186)
  ('US00724F1012', 'Electricity_prod_complete', NULL, 595461601.492059, 587480044.192059, 579498486.8920

In [16]:
# test that we can get data
qres = osc._do_sql(f'select y2020 from {ingest_catalog}.{ingest_schema}.{tablename}', engine, verbose=True)

select y2020 from osc_datacommons_dev.demo_dv.itr_projected_production
[(96880847.84303102,), (553531525.9021901,), (112320002.661096,), (92429383.29377028,), (745588806.472471,), (199591200.376334,), (444466802.585938,), (1360143531.0659223,), (19526537.978397347,), (934632000.90193,), (65804401.1549344,), (288420004.281372,), (44045949.40149568,), (528390001.129387,), (224471679.66002035,), (71500001.3960884,), (None,), (None,), (12194000.4760821,), (23652554.82218139,), (28090000.2335485,), (12819451.062523922,), (22329000.8292913,), (47050001.3676141,), (15752804.700495042,), (None,), (11314001.9224849,), (14473000.0778486,), (30630004.3464472,), (None,)]


## ITR Projected Target

In [17]:
tablename = f'{itr_prefix}projected_target'

# get this sheet, and assess data types
df = pd.read_excel(xls, 'projected_target').convert_dtypes()

# rename columns to forms that sql will handle
osc.enforce_sql_column_names(df, inplace=True)
df.rename(columns=rename_year_columns, inplace=True)

In [18]:
# For this demonstration example, we just recreate table from scratch each time.
# in live data platform there will need to be policies and mechanisms for either
# appending new data, or overwriting old data, or saving off conditioned by a versioning number
# this is a data governance topic
osc._do_sql(f'drop table if exists {ingest_catalog}.{ingest_schema}.{tablename}', engine, verbose=True)

drop table if exists osc_datacommons_dev.demo_dv.itr_projected_target


In [19]:
# generate a sql schema that will correspond to the data types
# of columns in the pandas DF

df.to_sql(tablename, engine, schema=ingest_schema,
          if_exists='append', index=False,
          method=osc.TrinoBatchInsert(batch_size = 15000, verbose = True))

constructed fully qualified table name as: "demo_dv.itr_projected_target"
inserting 60 records
  ('US0079031078', 'Intensity_scope1_complete', NULL, NULL, NULL, NULL, NULL, 0.170354939322627, 0.220299831720245, 0.187623393698818, 0.266897284395678, 0.501375798577897, 0.468310680717861, 0.468310680717861, 0.42928479065803926, 0.3902589005982175, 0.3785511335802709, 0.3668433665623244, 0.3551355995443779, 0.34342783252643133, 0.3317200655084848, 0.32001229849053825, 0.30830453147259174, 0.2965967644546452, 0.2831099480008877, 0.27544668915086834, 0.267220580884889, 0.25800218182173723, 0.24748360557364565, 0.23562865116810308, 0.2227383155461044, 0.2093270172783255, 0.19590974950985224, 0.18286671115048947, 0.17042259810933935, 0.15868382409266663, 0.14768299945780478, 0.13741184379498675, 0.12784145702501398, 0.11893370882207119, 0.11064730250250693, 0.10294087337164555, 0.09577449761700338, 0.08911036813081348)
  ('US0079031078', 'Intensity_scope2_complete', NULL, NULL, NULL, NULL, NUL

## ITR Projected ei in Wh

In [20]:
tablename = f'{itr_prefix}projected_ei_in_wh'

# get this sheet, and assess data types
df = pd.read_excel(xls, 'projected_ei_in_Wh').convert_dtypes()

# rename columns to forms that sql will handle
osc.enforce_sql_column_names(df, inplace=True)
df.rename(columns=rename_year_columns, inplace=True)

In [21]:
# For this demonstration example, we just recreate table from scratch each time.
# in live data platform there will need to be policies and mechanisms for either
# appending new data, or overwriting old data, or saving off conditioned by a versioning number
# this is a data governance topic
osc._do_sql(f'drop table if exists {ingest_catalog}.{ingest_schema}.{tablename}', engine, verbose=True)

drop table if exists osc_datacommons_dev.demo_dv.itr_projected_ei_in_wh


In [22]:
# generate a sql schema that will correspond to the data types
# of columns in the pandas DF

df.to_sql(tablename, engine, schema=ingest_schema,
          if_exists='append', index=False,
          method=osc.TrinoBatchInsert(batch_size = 15000, verbose = True))

constructed fully qualified table name as: "demo_dv.itr_projected_ei_in_wh"
inserting 60 records
  ('US0079031078', 'Intensity_scope1_complete', NULL, NULL, NULL, NULL, NULL, 0.170354939322627, 0.220299831720245, 0.187623393698818, 0.266897284395678, 0.501375798577897, 0.468310680717861, 0.468310680717861, 0.4384721079520032, 0.4111273932967205, 0.3863680753691475, 0.3643764337472672, 0.34537391807885887, 0.3294679702931648, 0.3164614050737526, 0.3058131940431168, 0.29681299030155467, 0.28878409012681017, 0.2811605800076067, 0.27346012212033577, 0.26523573184196947, 0.25607444019517955, 0.24568278207156138, 0.23402634921052445, 0.22138889013283639, 0.20825467980986892, 0.19510921053287353, 0.18231281067794972, 0.17008088864527465, 0.15851804007011908, 0.14765913625607005, 0.13749990482375027, 0.12801589692682325, 0.11917324380611401, 0.11093445963123792, 0.10326146613042127, 0.09611711472809159, 0.0894659114849519)
  ('US0079031078', 'Intensity_scope2_complete', NULL, NULL, NULL, NULL,